# Download Any OSM Geospatial Features with OSMnx

Author: [Geoff Boeing](https://geoffboeing.com/)

More info:

  - [Documentation](https://osmnx.readthedocs.io/)
  - [Journal article and citation info](https://doi.org/10.1111/gean.70009)
  - [Code repository](https://github.com/gboeing/osmnx)
  - [Examples gallery](https://github.com/gboeing/osmnx-examples)
  
This notebook provides a quick tour of using OSMnx to download any geospatial features from OpenStreetMap as a geopandas GeoDataFrame.

In [ ]:
#!uv pip install --system --quiet osmnx[all]
import osmnx as ox
import pandas as pd

ox.__version__

Use the `geometries` module to download features, such as grocery stores, transit stops, points of interest, or building footprints, and turn them into a GeoDataFrame: [see docs](https://osmnx.readthedocs.io/en/stable/osmnx.html#module-osmnx.geometries).

To query, pass a `tags` dict where keys are OSM tags. The dict's values can be either:
  1. `True` to retrieve all OSM objects with this tag, regardless of its value
  2. a single value as a string to retrieve all OSM objects with this tag:value combination
  3. a list of string values to retrieve all OSM objects where this tag matches any of these values

Pass multiple dict key:value pairs to retrieve the union (not intersection) of these pairs.

In [ ]:
# get all building footprints in some neighborhood
# `True` means retrieve any object with this tag, regardless of value
place = "Chinatown, San Francisco, California"
tags = {"building": True}
gdf = ox.features.features_from_place(place, tags)
gdf.shape

In [ ]:
fig, ax = ox.plot.plot_footprints(gdf, figsize=(3, 3))

In [ ]:
# get all the parks in some neighborhood
# constrain acceptable `leisure` tag values to `park`
tags = {"leisure": "park"}
gdf = ox.features.features_from_place(place, tags)
gdf.shape

In [ ]:
# get everything tagged amenity,
# and everything tagged landuse = retail or commercial,
# and everything tagged highway = bus_stop
tags = {"amenity": True, "landuse": ["retail", "commercial"], "highway": "bus_stop"}
gdf = ox.features.features_from_place("Piedmont, California, USA", tags)
gdf.shape

In [ ]:
# view just the banks
gdf[gdf["amenity"] == "bank"].dropna(axis=1, how="any")

In [ ]:
# view just the bus stops
gdf[gdf["highway"] == "bus_stop"].dropna(axis=1, how="any").head()

## Attach features to nearest nodes

For example, attach parking information to nearest network nodes.

In [ ]:
# first create the graph and the features
place = "Piedmont, CA, USA"
G = ox.graph.graph_from_place(place, network_type="drive")
features = ox.features.features_from_place(place, {"amenity": "parking"})

In [ ]:
# then attach your features to your graph's nearest nodes as attributes
feature_points = features.representative_point()
nn = ox.distance.nearest_nodes(G, feature_points.x, feature_points.y)
useful_tags = ["access", "parking", "surface", "capacity", "fee"]
for node, feature in zip(nn, features[useful_tags].to_dict(orient="records")):
    feature = {k: v for k, v in feature.items() if pd.notna(v)}
    G.nodes[node].update({"parking": feature})